# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 10:32AM,244469,22,624673,"NBTY Global, Inc.",Released
1,Aug 18 2022 10:29AM,244466,10,MSP213219,"Methapharm, Inc.",Released
2,Aug 18 2022 10:29AM,244465,21,406608781,"NBTY Global, Inc.",Released
3,Aug 18 2022 10:26AM,244463,10,8414220,Eywa Pharma Inc.,Released
4,Aug 18 2022 10:26AM,244463,10,8414292,Eywa Pharma Inc.,Released
5,Aug 18 2022 10:26AM,244463,10,8414293,Eywa Pharma Inc.,Released
6,Aug 18 2022 10:26AM,244463,10,8414294,Eywa Pharma Inc.,Released
7,Aug 18 2022 10:26AM,244463,10,8414295,Eywa Pharma Inc.,Released
8,Aug 18 2022 10:26AM,244463,10,8414297,Eywa Pharma Inc.,Released
9,Aug 18 2022 10:26AM,244463,10,8414298,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,244462,Released,1
48,244463,Released,29
49,244465,Released,1
50,244466,Released,1
51,244469,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244462,NaN,NaN,1.0
244463,NaN,NaN,29.0
244465,NaN,NaN,1.0
244466,NaN,NaN,1.0
244469,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,18.0,2.0,0.0
244357,9.0,2.0,2.0
244370,0.0,0.0,1.0
244385,11.0,4.0,0.0
244396,0.0,0.0,73.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,18,2,0
244357,9,2,2
244370,0,0,1
244385,11,4,0
244396,0,0,73


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,18,2,0
1,244357,9,2,2
2,244370,0,0,1
3,244385,11,4,0
4,244396,0,0,73


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,18,2,
1,244357,9,2,2
2,244370,,,1
3,244385,11,4,
4,244396,,,73


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc."
1,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc."
2,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc."
3,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.
32,Aug 18 2022 10:22AM,244462,10,Emerginnova
33,Aug 18 2022 10:19AM,244461,10,"Snap Medical Industries, LLC"
35,Aug 18 2022 10:16AM,244460,20,"ACI Healthcare USA, Inc."
36,Aug 18 2022 10:11AM,244459,20,"ACI Healthcare USA, Inc."
53,Aug 18 2022 9:50AM,244444,20,Alumier Labs Inc.
65,Aug 18 2022 9:50AM,244455,17,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",,,1
1,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",,,1
2,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",,,1
3,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,,,29
4,Aug 18 2022 10:22AM,244462,10,Emerginnova,,,1
5,Aug 18 2022 10:19AM,244461,10,"Snap Medical Industries, LLC",,,2
6,Aug 18 2022 10:16AM,244460,20,"ACI Healthcare USA, Inc.",,,1
7,Aug 18 2022 10:11AM,244459,20,"ACI Healthcare USA, Inc.",,,17
8,Aug 18 2022 9:50AM,244444,20,Alumier Labs Inc.,,2,10
9,Aug 18 2022 9:50AM,244455,17,Beach Products Inc,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",1,,
1,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",1,,
2,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",1,,
3,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,29,,
4,Aug 18 2022 10:22AM,244462,10,Emerginnova,1,,
5,Aug 18 2022 10:19AM,244461,10,"Snap Medical Industries, LLC",2,,
6,Aug 18 2022 10:16AM,244460,20,"ACI Healthcare USA, Inc.",1,,
7,Aug 18 2022 10:11AM,244459,20,"ACI Healthcare USA, Inc.",17,,
8,Aug 18 2022 9:50AM,244444,20,Alumier Labs Inc.,10,2,
9,Aug 18 2022 9:50AM,244455,17,Beach Products Inc,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",1,,
1,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",1,,
2,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",1,,
3,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,29,,
4,Aug 18 2022 10:22AM,244462,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,29.0,NaN,NaN
4,Aug 18 2022 10:22AM,244462,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,29.0,0.0,0.0
4,Aug 18 2022 10:22AM,244462,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4155404,155.0,32.0,9.0
102,244450,1.0,0.0,0.0
12,488870,1.0,14.0,0.0
16,733287,21.0,0.0,0.0
17,488861,2.0,0.0,0.0
19,733234,20.0,11.0,18.0
20,1466526,105.0,4.0,18.0
21,1466560,6.0,0.0,0.0
22,244469,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4155404,155.0,32.0,9.0
1,102,244450,1.0,0.0,0.0
2,12,488870,1.0,14.0,0.0
3,16,733287,21.0,0.0,0.0
4,17,488861,2.0,0.0,0.0
5,19,733234,20.0,11.0,18.0
6,20,1466526,105.0,4.0,18.0
7,21,1466560,6.0,0.0,0.0
8,22,244469,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,155.0,32.0,9.0
1,102,1.0,0.0,0.0
2,12,1.0,14.0,0.0
3,16,21.0,0.0,0.0
4,17,2.0,0.0,0.0
5,19,20.0,11.0,18.0
6,20,105.0,4.0,18.0
7,21,6.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,155.0
1,102,Released,1.0
2,12,Released,1.0
3,16,Released,21.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,17,19,20,21,22
Status,,,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
Executing,32.0,0.0,14.0,0.0,0.0,11.0,4.0,0.0,0.0
Released,155.0,1.0,1.0,21.0,2.0,20.0,105.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,17,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
1,Executing,32.0,0.0,14.0,0.0,0.0,11.0,4.0,0.0,0.0
2,Released,155.0,1.0,1.0,21.0,2.0,20.0,105.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,17,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
1,Executing,32.0,0.0,14.0,0.0,0.0,11.0,4.0,0.0,0.0
2,Released,155.0,1.0,1.0,21.0,2.0,20.0,105.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()